# [Module 9] 리소스 삭제

모델을 구축한 후 캠페인(campaign), 솔루션(solution) 및 데이터셋(dataset)을 삭제할 수 있습니다. 다음 셀은 이 핸즈온에서 생성한 모든 리소스를 삭제합니다.

## 라이브러리 임포트 및 AWS 연결 

아래 코드 셀은 필요한 모든 라이브러리를 가져온 다음 Amazon Personalize에 연결합니다.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
%store -r

# 1. 리소스 삭제 함수 정의

In [ ]:
def wait_for_resource_deletion(arn, resource_type):
    max_time = 600  # 10분 타임아웃
    wait_time = 0
    
    while wait_time < max_time:
        try:
            if resource_type == 'campaign':
                response = personalize.describe_campaign(campaignArn=arn)
            elif resource_type == 'solution':
                response = personalize.describe_solution(solutionArn=arn)
            elif resource_type == 'dataset':
                response = personalize.describe_dataset(datasetArn=arn)
            elif resource_type == 'schema':
                response = personalize.describe_schema(schemaArn=arn)
            elif resource_type == 'dataset-group':
                response = personalize.describe_dataset_group(datasetGroupArn=arn)
                
            print(f"Waiting for {resource_type} deletion... Status: {response['status']}")
            time.sleep(10)
            wait_time += 10
            
        except personalize.exceptions.ResourceNotFoundException:
            print(f"{resource_type} deleted successfully")
            return True
            
    raise Exception(f"Timeout waiting for {resource_type} deletion")

def cleanup_personalize_resources(base_campaign_arn, base_solution_arn, 
                                base_interaction_dataset_arn, base_item_dataset_arn, base_user_dataset_arn,
                                interaction_schema_arn, base_item_schema_arn, user_schema_arn,
                                base_dataset_group_arn):
    try:
        # 1. Campaign 삭제
        print("Deleting campaign...")
        personalize.delete_campaign(campaignArn=base_campaign_arn)
        wait_for_resource_deletion(base_campaign_arn, 'campaign')

        # 2. Solution 삭제
        print("Deleting solution...")
        personalize.delete_solution(solutionArn=base_solution_arn)
        wait_for_resource_deletion(base_solution_arn, 'solution')

        # 3. Datasets 삭제
        print("Deleting datasets...")
        dataset_arns = [base_interaction_dataset_arn, base_item_dataset_arn, base_user_dataset_arn]
        for dataset_arn in dataset_arns:
            personalize.delete_dataset(datasetArn=dataset_arn)
            wait_for_resource_deletion(dataset_arn, 'dataset')

        # 4. Schemas 삭제
        print("Deleting schemas...")
        schema_arns = [interaction_schema_arn, base_item_schema_arn, user_schema_arn]
        for schema_arn in schema_arns:
            personalize.delete_schema(schemaArn=schema_arn)
            wait_for_resource_deletion(schema_arn, 'schema')

        # 5. Dataset Group 삭제
        print("Deleting dataset group...")
        personalize.delete_dataset_group(datasetGroupArn=base_dataset_group_arn)
        wait_for_resource_deletion(base_dataset_group_arn, 'dataset-group')

        print("All resources cleaned up successfully!")

    except Exception as e:
        print(f"Error during cleanup: {str(e)}")
        raise

# 2. 리소스 삭제

In [ ]:
cleanup_personalize_resources(
    base_campaign_arn,
    base_solution_arn,
    base_interaction_dataset_arn,
    base_item_dataset_arn,
    base_user_dataset_arn,
    interaction_schema_arn,
    base_item_schema_arn,
    user_schema_arn,
    base_dataset_group_arn
)

# 3. S3 파일(csv) 삭제

In [ ]:
s3_path = "s3://{}/{}/".format(bucket, 'base-dataset')

! aws s3 rm {s3_path} --recursive

### 디스크 저장 변수 삭제

디스크에 저장된 변수들을 모두 지웁니다.

In [ ]:
%store -z